# 🎯 Monitor Inteligente de Opinión: Trustpilot Unified
## 👥 Proyecto End-to-End de Data Science - Herramienta de Inteligencia de Mercado

Este notebook unifica las mejores funcionalidades de los prototipos anteriores para ofrecer una herramienta robusta de análisis de sentimiento comercial.

### ✨ Novedades y Autoría:
1.  **Búsqueda Dinámica (Rubén)**: Introduce el nombre de una empresa y el scraper encontrará el enlace automáticamente.
2.  **Modo Stealth Pro (Rubén)**: Integración de rotación de User-Agents y retardos aleatorios para evitar bloqueos.
3.  **Análisis Combinado (Rubén)**: NLP avanzado con diccionarios locales de español y modelos de polaridad.
4.  **BI Dashboard (Juanes/Rubén)**: Visualizaciones interactivas y estáticas para reportes de negocio.

**Metodología:**
1. 📥 **FASE 1**: Adquisición de Datos (Web Scraping Inteligente)
2. 🧹 **FASE 2**: Preprocesamiento NLP (Limpieza y Lematización)
3. 💎 **FASE 3**: Extracción de Valor (Sentimiento y Frecuencia)
4. 📊 **FASE 4**: Visualización de Impacto (BI)

In [ ]:
# =============================================================================
# FASE 0: CONFIGURACIÓN DEL ENTORNO
# [Original + Mejora de Compatibilidad Windows: Rubén]
# =============================================================================
import sys
import subprocess

print("🔧 Configurando entorno de ejecución...")

def install_packages():
    packages = [
        "requests", "beautifulsoup4", "lxml", "nltk", "textblob", 
        "googletrans==4.0.0-rc1", "wordcloud", "matplotlib", 
        "seaborn", "plotly", "fake-useragent", "pandas", 
        "numpy", "regex", "tqdm", "spacy"
    ]
    
    print("📦 Instalando librerías necesarias (esto puede tardar unos minutos)...")
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        except Exception as e:
            print(f"⚠️ Error al instalar {package}: {e}")
    
    # Descargar modelo de Spacy para español
    try:
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_sm", "-q"])
    except:
        print("⚠️ No se pudo descargar el modelo de spacy automáticamente.")

install_packages()

import nltk
import spacy

print("📥 Descargando recursos lingüísticos...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

print("✅ Entorno listo.")

In [ ]:
# [Original + Imports Rubén para funcionalidad Stealth]
import pandas as pd
import numpy as np
import requests
import re
import time
import random
import json
from datetime import datetime
from typing import List, Dict, Optional
from collections import Counter
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from textblob import TextBlob
from googletrans import Translator
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8')
sns.set_palette("deep")
print("📚 Librerías importadas correctamente.")

# 📥 FASE 1: ADQUISICIÓN DE DATOS (SCRAPER INTELIGENTE)

In [ ]:
class TrustpilotScraper:
    """
    Scraper avanzado mejorado para robustez y anonimato.
    [Base: Corrección_TrustPilot | Mejoras: Rubén]
    """
    def __init__(self, business_query: str = None, max_pages: int = 5):
        self.query = business_query
        self.max_pages = max_pages
        self.ua = UserAgent() # [Rubén] Integración de UA dinámica
        self.session = requests.Session()
        self.base_url = "https://es.trustpilot.com"
        self.target_url = None
        self.reviews_data = []

    def _get_headers(self):
        """[Rubén] Genera headers aleatorios para cada petición (Modo Stealth)."""
        return {
            'User-Agent': self.ua.random,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive'
        }

    def search_business(self) -> bool:
        """[Rubén] Busca la URL de la empresa automáticamente."""
        if not self.query:
            return False
        
        if "trustpilot.com/review/" in self.query:
            self.target_url = self.query
            return True

        print(f"🔍 Buscando empresa: '{self.query}' en Trustpilot...")
        search_url = f"{self.base_url}/search?query={self.query.replace(' ', '+')}"
        
        try:
            response = self.session.get(search_url, headers=self._get_headers(), timeout=15)
            soup = BeautifulSoup(response.content, 'html.parser')
            link_element = soup.select_one('a[data-business-unit-card-link="true"]')
            if link_element and link_element.has_attr('href'):
                path = link_element['href']
                self.target_url = self.base_url + path
                print(f"✅ Enlace encontrado: {self.target_url}")
                return True
            else:
                print("❌ No se encontró la empresa. Intenta con la URL directa.")
                return False
        except Exception as e:
            print(f"⚠️ Error durante la búsqueda: {e}")
            return False

    def safe_extract(self, element, select_list: List[str], attr: str = None) -> str:
        """[Base: Corrección_TrustPilot] Prueba múltiples selectores para extraer un dato."""
        for selector in select_list:
            found = element.select_one(selector)
            if found:
                if attr:
                    return found.get(attr, "")
                return found.get_text(strip=True)
        return ""

    def extract_review_info(self, card) -> Optional[Dict]:
        """Extrae los campos de una tarjeta de reseña individual. [Base: Corrección | Mejoras Fecha: Rubén]"""
        txt_selectors = [
            'p[data-service-review-text-typography="true"]',
            'p[data-relevant-review-text-typography="true"]',
            'p[data-review-content-typography="true"]',
            '.styles_reviewContent__0_vST p'
        ]
        
        star_selectors = ['.styles_reviewHeader__iU9_n img', 'div[data-star-rating]']
        date_selectors = ['time', 'span[data-service-review-date-time-ago]']

        text = self.safe_extract(card, txt_selectors)
        rating_raw = self.safe_extract(card, star_selectors, 'alt')
        rating = re.search(r'\d', rating_raw).group() if re.search(r'\d', rating_raw) else "0"
        date_raw = self.safe_extract(card, date_selectors, 'datetime')
        
        # [Rubén] Manejo avanzado de fechas ISO
        try:
            if date_raw:
                date_obj = datetime.fromisoformat(date_raw.replace("Z", "+00:00"))
                date_str = date_obj.strftime('%Y-%m-%d')
            else:
                date_str = datetime.now().strftime('%Y-%m-%d')
        except:
            date_str = datetime.now().strftime('%Y-%m-%d')

        if not text or len(text) < 10:
            return None

        return {
            'texto': text,
            'puntuacion': int(rating),
            'fecha': date_str,
            'longitud': len(text.split())
        }

    def run(self) -> pd.DataFrame:
        """[Integración Rubén: Flujo completo con delays aleatorios]"""
        if not self.target_url and not self.search_business():
            return pd.DataFrame()

        print(f"🚀 Iniciando extracción desde: {self.target_url}")
        
        for page in range(1, self.max_pages + 1):
            url = f"{self.target_url}?page={page}"
            print(f"📄 Procesando página {page}/{self.max_pages}...")
            
            try:
                # [Rubén] Delay aleatorio antidetect
                time.sleep(random.uniform(2, 4))
                
                response = self.session.get(url, headers=self._get_headers(), timeout=20)
                if response.status_code != 200:
                    print(f"⚠️ Error {response.status_code} al acceder a la página {page}")
                    break

                soup = BeautifulSoup(response.content, 'html.parser')
                cards = soup.select('article[data-service-review-card-paper="true"]')
                
                if not cards:
                    print("🏁 No se encontraron más reseñas.")
                    break
                
                for card in cards:
                    data = self.extract_review_info(card)
                    if data:
                        self.reviews_data.append(data)
                
                print(f"✅ {len(cards)} elementos analizados. Total acumulado: {len(self.reviews_data)}")
                
            except Exception as e:
                print(f"❌ Error crítico en página {page}: {e}")
                break
        
        df = pd.DataFrame(self.reviews_data)
        if not df.empty:
            df = df.drop_duplicates(subset=['texto'])
            print(f"\n📊 Extracción finalizada: {len(df)} reseñas únicas obtenidas.")
        return df

In [ ]:
# [Rubén: Interfaz de entrada dinámica]
print("🔎 CONFIGURACIÓN DE BÚSQUEDA")
print("Escribe el nombre de la empresa (ej: 'Amazon Spain') o pega la URL de Trustpilot:")
target = input("> ").strip() 

if not target:
    target = "Amazon Spain" 
    print(f"Usando valor por defecto: {target}")

# [Juanes: Lógica de ejecución del scraper]
scraper = TrustpilotScraper(business_query=target, max_pages=3)
df_raw = scraper.run()

if df_raw.empty:
    print("🔄 Cargando dataset de respaldo para demostración...")
    df_raw = pd.DataFrame({
        'texto': [
            "Excelente servicio, el paquete llegó antes de lo esperado. Muy contento.",
            "Pésima atención al cliente. Mi pedido nunca llegó y nadie responde.",
            "Producto de buena calidad, pero el envío es algo lento.",
            "No recomiendo comprar aquí. Me enviaron algo roto y no hay reembolso.",
            "Amazon sigue siendo el mejor, aunque a veces los repartidores fallan.",
            "Servicio técnico inexistente, una odisea para devolver un producto."
        ] * 10,
        'puntuacion': [5, 1, 4, 1, 4, 2] * 10,
        'fecha': pd.date_range(end=datetime.now(), periods=60).strftime('%Y-%m-%d').tolist(),
        'longitud': [10, 15, 12, 14, 11, 13] * 10
    })

df_raw.to_csv('reseñas_trustpilot_raw.csv', index=False, encoding='utf-8-sig')
display(df_raw.head())

# 🧹 FASE 2: PREPROCESAMIENTO NLP

In [ ]:
class SpanishTextPreprocessor:
    """
    [Base: MONITOR_INTELIGENCIA_OPINION_v14 | Adaptación: Rubén]
    Lógica de limpieza profunda de texto para análisis temático.
    """
    def __init__(self):
        # [Juanes: Carga inicial de stopwords locales y dominio]
        self.stopwords_es = set(stopwords.words('spanish'))
        self.extra_stopwords = {
            'amazon', 'bueno', 'malo', 'hacer', 'decir', 'ver', 'querer', 
            'cliente', 'pedido', 'producto', 'servicio', 'envío', 'compra'
        }
        self.stopwords_es.update(self.extra_stopwords)
        try:
            self.nlp = spacy.load("es_core_news_sm")
        except:
            self.nlp = None

    def clean(self, text: str) -> str:
        """[Original: Limpieza básica]"""
        if not isinstance(text, str): return ""
        text = text.lower()
        text = re.sub(r'[^a-záéíóúüñ\s]', ' ', text)
        text = ' '.join(text.split())
        return text

    def remove_stopwords(self, text: str) -> str:
        """[Juanes: Filtrado selectivo]"""
        tokens = word_tokenize(text)
        return ' '.join([w for w in tokens if w not in self.stopwords_es and len(w) > 2])

    def process_pipeline(self, text: str) -> str:
        cleaned = self.clean(text)
        no_stop = self.remove_stopwords(cleaned)
        return no_stop

print("🧼 Procesando textos...")
preprocessor = SpanishTextPreprocessor()
df_processed = df_raw.copy()
df_processed['texto_limpio'] = [preprocessor.process_pipeline(t) for t in tqdm(df_processed['texto'])]
print("✅ Preprocesamiento completado.")

# 💎 FASE 3: ANÁLISIS DE SENTIMIENTO Y VALOR

In [ ]:
class SentimentAnalyzerES:
    """
    IA de Sentimiento Híbrida.
    [Base: MONITOR_INTELIGENCIA_OPINION_v14 | Novedad Híbrida: Rubén]
    """
    def __init__(self):
        self.pos_words = {'excelente', 'perfecto', 'genial', 'rápido', 'bueno', 'feliz', 'contento', 'recomiendo', 'útil', 'eficiente'}
        self.neg_words = {'pésimo', 'horrible', 'terrible', 'error', 'malo', 'estafa', 'fraude', 'inútil', 'lento', 'decepción'}
        self.translator = Translator() # [Rubén] Traductor para modelo superior

    def analyze_simple(self, text: str) -> Dict:
        """[Juanes: Método por diccionario local]"""
        words = text.split()
        pos = sum(1 for w in words if w in self.pos_words)
        neg = sum(1 for w in words if w in self.neg_words)
        total = pos + neg
        if total == 0: return {'score': 0, 'label': 'neutral', 'conf': 0}
        score = (pos - neg) / total
        label = 'positivo' if score > 0.1 else ('negativo' if score < -0.1 else 'neutral')
        return {'score': score, 'label': label, 'conf': abs(score)}

    def analyze_textblob(self, text: str) -> Dict:
        """[Rubén: Análisis profundo vía traducción]"""
        try:
            translated = self.translator.translate(text, src='es', dest='en').text
            analysis = TextBlob(translated)
            score = analysis.sentiment.polarity
            label = 'positivo' if score > 0.1 else ('negativo' if score < -0.1 else 'neutral')
            return {'score': score, 'label': label, 'conf': abs(score)}
        except:
            return self.analyze_simple(text)

    def combined(self, text: str) -> Dict:
        """[Rubén: Orquestador Híbrido]"""
        res = self.analyze_simple(text)
        # Si el texto es largo y el diccionario es débil, usar IA profunda
        if res['label'] == 'neutral' and len(text) > 40:
            return self.analyze_textblob(text)
        return res

print("😊 Aplicando IA de sentimiento...")
analyzer = SentimentAnalyzerES()
sentiments = [analyzer.combined(t) for t in tqdm(df_processed['texto_limpio'])]

df_processed['sentimiento_score'] = [s['score'] for s in sentiments]
df_processed['sentimiento'] = [s['label'] for s in sentiments]
df_processed['confianza'] = [s['conf'] for s in sentiments]

print(f"\n📊 RESUMEN DE SENTIMIENTO:")
print(df_processed['sentimiento'].value_counts(normalize=True) * 100)
df_processed.to_csv('reseñas_trustpilot_final.csv', index=False, encoding='utf-8-sig')

# 📊 FASE 4: VISUALIZACIÓN GRÁFICA (BI)

In [ ]:
# [Diseño base de Dashboards: Juanes | Enriquecimiento Temporal: Rubén]

# 1. Configuración de Gráficos
fig, axes = plt.subplots(3, 2, figsize=(18, 18))
plt.subplots_adjust(hspace=0.4)

# ☁️ Gráfico 1: Nube de Palabras [Juanes]
all_text = ' '.join(df_processed['texto_limpio'])
if all_text:
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='coolwarm').generate(all_text)
    axes[0, 0].imshow(wordcloud, interpolation='bilinear')
    axes[0, 0].set_title('🌥️ Temas más mencionados [Juanes]', fontsize=15)
    axes[0, 0].axis('off')

# 📊 Gráfico 2: Distribución de Sentimiento [Juanes]
colors = {'positivo': '#2ecc71', 'negativo': '#e74c3c', 'neutral': '#f1c40f'}
sns.countplot(data=df_processed, x='sentimiento', palette=colors, ax=axes[0, 1])
axes[0, 1].set_title('📉 Salud de Marca (Distribución) [Juanes]', fontsize=15)

# 📈 Gráfico 3: Correlación Longitud vs Sentimiento [Juanes]
sns.regplot(data=df_processed, x='longitud', y='sentimiento_score', ax=axes[1, 0], 
            scatter_kws={'alpha':0.5, 'color':'teal'}, line_kws={'color':'red'})
axes[1, 0].set_title('📏 Detalle de Reseña vs Score [Juanes]', fontsize=15)

# 🏷️ Gráfico 4: Top 10 Palabras [Juanes]
top_words = Counter(all_text.split()).most_common(10)
words_df = pd.DataFrame(top_words, columns=['palabra', 'frecuencia'])
sns.barplot(data=words_df, x='frecuencia', y='palabra', ax=axes[1, 1], palette='viridis')
axes[1, 1].set_title('🏆 Top 10 Palabras Clave [Juanes]', fontsize=15)

# 🕒 Gráfico 5: Evolución Temporal del Sentimiento [Rubén]
df_processed['fecha'] = pd.to_datetime(df_processed['fecha'])
temporal = df_processed.groupby('fecha')['sentimiento_score'].mean().reset_index()
sns.lineplot(data=temporal, x='fecha', y='sentimiento_score', ax=axes[2, 0], marker='o', color='purple')
axes[2, 0].axhline(0, color='black', linestyle='--', alpha=0.3)
axes[2, 0].set_title('🕒 Evolución de la Opinión (Tendencias) [Rubén]', fontsize=15)
axes[2, 0].tick_params(axis='x', rotation=45)

# 🌟 Gráfico 6: Heatmap Estrellas vs Sentimiento [Rubén]
matrix = pd.crosstab(df_processed['puntuacion'], df_processed['sentimiento'])
sns.heatmap(matrix, annot=True, fmt='d', cmap='YlGnBu', ax=axes[2, 1])
axes[2, 1].set_title('🔥 Consistencia Estrellas vs IA [Rubén]', fontsize=15)

plt.show()